In [4]:
import pandas as pd
import os

In [18]:
working_dir = os.getcwd()
rel_path = r"project_resources\chembl_molekuly.csv"
abs_file_path = os.path.join(working_dir, rel_path)

In [19]:
df = pd.read_csv(abs_file_path, sep=";")
df.head()

C:\Users\Lukas\AppData\Local\Temp\ipykernel_14064\2078184646.py:1: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(abs_file_path, sep=";")


,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Target Name,Target Organism,Target Type,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties
0,CHEMBL2373647,NaN,None,554.68,2,5.75,SID26749372,CCC(C)C(=O)OC1C2CC3=C4CC(=O)OC(c5ccoc5)[C@]4(C...,Potency,'=',...,Cytochrome P450 3A4,Homo sapiens,SINGLE PROTEIN,CHEMBL1201862,7,PubChem BioAssays,NaN,NaN,None,NaN
1,CHEMBL1396146,NaN,None,436.42,0,4.10,SID4239234,O=c1c(-c2cc(F)cc(F)c2)nc2cnc(Oc3ccccc3)nc2n1C[...,Potency,'=',...,Cytochrome P450 3A4,Homo sapiens,SINGLE PROTEIN,CHEMBL1201862,7,PubChem BioAssays,NaN,NaN,None,NaN
2,CHEMBL1554982,NaN,None,595.70,1,4.71,SID17385173,COC(=O)[C@@]1(Cc2ccc(OC)cc2)[C@H]2c3cc(C(=O)N(...,Potency,'=',...,Cytochrome P450 3A4,Homo sapiens,SINGLE PROTEIN,CHEMBL1201862,7,PubChem BioAssays,NaN,NaN,None,NaN
3,CHEMBL1590886,NaN,None,424.56,0,2.78,SID4240445,CN1CCN(c2ncc3nc(-c4cccs4)c(=O)n(Cc4cccs4)c3n2)CC1,Potency,'=',...,Cytochrome P450 3A4,Homo sapiens,SINGLE PROTEIN,CHEMBL1201862,7,PubChem BioAssays,NaN,NaN,None,NaN
4,CHEMBL16687,CGS-15943,None,285.69,0,2.77,SID11114185,Nc1nc2ccc(Cl)cc2c2nc(-c3ccco3)nn12,Potency,'=',...,Cytochrome P450 3A4,Homo sapiens,SINGLE PROTEIN,CHEMBL1201862,7,PubChem BioAssays,NaN,NaN,None,NaN


In [20]:
import pandas as pd
df = pd.read_csv(abs_file_path, sep=";")
statistika = df["Standard Type"].value_counts()
print(statistika)

AC50                          17143
Potency                       13692
Activity                       1602
Ki                             1112
Drug metabolism                 498
FC                              195
Stability                       126
Kinact                          111
Ratio                            97
Km                               71
T1/2                             70
K                                66
Log 1/Km                         58
Compound recovery                54
CL                               45
Kcat                             44
EC50                             36
mechanism based inhibition       34
Ks                               31
Vmax                             28
Rate of oxidation                18
Kd                               16
Delta Tm                         16
Vmax/Km                          14
IC20                             13
Emax                             12
Retention_time                   12
Inhibitory index            

C:\Users\Lukas\AppData\Local\Temp\ipykernel_14064\411097526.py:2: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(abs_file_path, sep=";")


In [22]:
result = df[(df["Standard Type"] == "t1/2") | (df["Standard Type"] == "T1/2")]
t_half_rel_path = r"project_resources\t_half.csv"
t_half_abs_path = os.path.join(script_dir, t_half_rel_path)
result.to_csv(t_half_abs_path, index=False)
#creates "t_half.csv" in current dir

In [ ]:
from chembl_webresource_client.new_client import new_client

client = new_client

#Specify the Chembl ID you want to convert
chembl_id = "CHEMBL1215730"

#molecule=new_client.molecule

#Use the client to retrieve the compound information
compound = new_client.molecule.filter(chembl_id=chembl_id)[0]

#Extract the SMILES string from the compound information
smiles = compound["molecule_structures"]["canonical_smiles"]

#Print the SMILES string
print(smiles)

In [ ]:
#Get the calculated properties of the compound
calculated_properties = compound['molecule_properties']

#Print out the calculated properties
for prop in calculated_properties: 
    print(prop, compound['molecule_properties'][prop])